## Compare results for multiple results

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [39]:
%matplotlib widget

In [40]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [41]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [42]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=64
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_err'][:idx]-dict_sample['spec_err'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict


In [43]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
s_val=np.load(fname,mmap_mode='r')[10000:18000][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [44]:
df_runs=pd.DataFrame([])

In [45]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:500][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [46]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [47]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201027_200951_with_spec_loss/'
prefix='pytorch_1_'

lst=[(2, 6662), (3, 11089), (11, 34800), (11, 35200), (12, 37800), (14, 44800)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(2000, 128, 128)
0.97673374 -0.9999999
(2000, 128, 128)
0.99551094 -0.9999965
(64, 128, 128)
0.9897282 -0.99929684
(64, 128, 128)
0.9866726 -0.9990417
(64, 128, 128)
0.99498445 -0.9996204
(64, 128, 128)
0.9931596 -0.9995343


In [48]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201107_180222_full_loss_b64_no-invtransform/'
prefix='pytorch_2_'

lst=[(29, 91944), (27, 85848), (25, 78324), (27, 86903)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)




(64, 128, 128)
0.99366 -0.9986709
(64, 128, 128)
0.9854231 -0.9989682
(64, 128, 128)
0.97410965 -0.99863833
(64, 128, 128)
0.99086607 -0.9991402


In [49]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201107_132246_full_loss_b64_withinvtransform/'
prefix='pytorch_3_'

lst=[(1, 6055), (22, 69260), (15, 46922), (11, 34772)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)




(64, 128, 128)
0.9813049 -0.9958283
(64, 128, 128)
0.9637603 -0.9996908
(64, 128, 128)
0.95702213 -0.9999663
(64, 128, 128)
0.92941725 -0.9997804


In [50]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')

In [51]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'chi_spec3','num_img']
df_runs=df_runs.reindex(columns=col_list)


In [52]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,chi_spec3,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0028445197484184732, 0.0020816260811514454,...","[1.121347563774932, 0.9576931243390049, 0.2568...","[1987.871939779571, 1009.5002022488515, 686.43...","[87592.048827034, 62024.131669237926, 44898.48...",0.014575,0.000112,0.000187,0.014276,0.000020,4.080889,0.001910,56.011413,20.162067,500.0
1,keras_0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003081765924486072, 0.0023479649643570986, ...","[1.1146491994998429, 0.9643402305198981, 0.260...","[1913.65866089898, 963.0815281971833, 644.6720...","[85906.63125591779, 60940.73459686716, 43971.7...",0.001702,0.000073,0.000449,0.001180,0.000055,4.072555,0.052795,2672.641410,22.488433,500.0
2,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0031456442853499706, 0.002349065715521853, ...","[1.1148820753244182, 0.9669777732485703, 0.261...","[2012.174777339885, 1015.3485395616655, 727.33...","[86439.30616832257, 60426.54691713405, 45079.8...",0.002058,0.000455,0.000231,0.001373,0.000113,4.171513,0.066858,2934.273632,22.173374,500.0
3,keras_2,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0032299157726108983, 0.00241667307354839, 0...","[1.1439634352627102, 0.9270007224057393, 0.253...","[1877.6099696345757, 1000.744571148909, 636.93...","[82519.15650125599, 59058.62442671653, 43854.1...",0.003165,0.002139,0.000352,0.000674,0.001705,4.130929,0.020309,621.498995,24.126142,500.0
4,keras_3,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0029680794378720322, 0.002208163391797882, ...","[1.0973298711470636, 0.9833819747814072, 0.269...","[2032.9510454630201, 978.7139317701136, 649.82...","[88968.60526771784, 62091.84879396162, 45370.3...",0.003478,0.001618,0.000507,0.001353,0.001206,4.001797,0.047763,2551.799936,20.860664,500.0
5,pytorch_1_2-6662,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002191719527429301, 0.0022940040381239854, ...","[1.4614886006859085, 0.5568758505816295, 0.168...","[707.1739753649352, 449.9077707534787, 294.597...","[48316.27567767973, 38695.251164858084, 30797....",0.565152,0.400419,0.152728,0.012005,0.294867,1.615331,15.742224,580068.234549,24.957677,2000.0
6,pytorch_1_3-11089,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.001587930930108506, 0.001406133000854355, 0...","[1.1896787740875971, 0.7703692429011583, 0.292...","[1861.4616830608693, 1002.388646599244, 483.54...","[93082.67361517194, 78453.05279839202, 60422.8...",0.151757,0.107378,0.022255,0.022125,0.045627,2.417146,3.611361,308707.589729,27.080441,2000.0
7,pytorch_1_11-34800,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.016823394472101422, 0.014705840140681823, 0...","[1.1712546974829157, 0.8750739111074888, 0.266...","[6306.421177965896, 2607.3966929839594, 1449.9...","[76970.52280858159, 54471.48822243213, 38839.9...",0.029449,0.011806,0.009205,0.008438,0.010006,14.863315,0.718692,1229.065931,28.570891,64.0
8,pytorch_1_11-35200,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.010905487760499852, 0.006638432859334996, 0...","[1.1312873690698122, 0.9086202993084255, 0.283...","[6679.864919736757, 4451.214280741923, 2877.25...","[88752.88910913467, 66932.44186913669, 47176.9...",0.027141,0.009368,0.011022,0.006752,0.003830,12.220433,0.486636,1850.241459,27.145761,64.0
9,pytorch_1_12-37800,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975

### Compare chi-squares

In [53]:
chi_keys=['label','chi_1','chi_spec1', 'num_img', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_img,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,raw,0.014575,0.001910,500.0,0.000112,0.000187,0.014276,0.000020,4.080889,56.011413,20.162067
1,keras_0,0.001702,0.052795,500.0,0.000073,0.000449,0.001180,0.000055,4.072555,2672.641410,22.488433
2,keras_1,0.002058,0.066858,500.0,0.000455,0.000231,0.001373,0.000113,4.171513,2934.273632,22.173374
3,keras_2,0.003165,0.020309,500.0,0.002139,0.000352,0.000674,0.001705,4.130929,621.498995,24.126142
4,keras_3,0.003478,0.047763,500.0,0.001618,0.000507,0.001353,0.001206,4.001797,2551.799936,20.860664
5,pytorch_1_2-6662,0.565152,15.742224,2000.0,0.400419,0.152728,0.012005,0.294867,1.615331,580068.234549,24.957677
6,pytorch_1_3-11089,0.151757,3.611361,2000.0,0.107378,0.022255,0.022125,0.045627,2.417146,308707.589729,27.080441
7,pytorch_1_11-34800,0.029449,0.718692,64.0,0.011806,0.009205,0.008438,0.010006,14.863315,1229.065931,28.570891
8,pytorch_1_11-35200,0.027141,0.486636,64.0,0.009368,0.011022,0.006752,0.003830,12.220433,1850.241459,27.145761
9,pytorch_1_12-37800,0.031397,0.743896,64.0,0.007525,0.019528,0.004344,0.005195,9.086633,3849.303141,27.134783


## Plot

In [54]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [55]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [32]:
col_list=['label','chi_1','chi_spec1','num_img','chi_2','chi_spec2']
df_runs[col_list].head(10)

,label,chi_1,chi_spec1,num_img,chi_2,chi_spec2
0,raw,0.014575,0.001910,500.0,0.000020,56.011413
1,keras_0,0.001702,0.052795,500.0,0.000055,2672.641410
2,keras_1,0.002058,0.066858,500.0,0.000113,2934.273632
3,keras_2,0.003165,0.020309,500.0,0.001705,621.498995
4,keras_3,0.003478,0.047763,500.0,0.001206,2551.799936
5,pytorch_1_2-6662,0.565152,15.742224,2000.0,0.294867,580068.234549
6,pytorch_1_3-11089,0.151757,3.611361,2000.0,0.045627,308707.589729
7,pytorch_1_11-34800,0.029449,0.718692,64.0,0.010006,1229.065931
8,pytorch_1_11-35200,0.027141,0.486636,64.0,0.003830,1850.241459
9,pytorch_1_12-37800,0.031397,0.743896,64.0,0.005195,3849.303141
